# Linear Regression

Los modelos de regresión son como 'calculadoras inteligentes' que usan datos pasados para predecir números. Por esta razon usaremo un ejemplo basado en un estudio real en el que se recopilaron datos de un sistema de bicicletas compartidas para predecir el número de alquileres en función de la estacionalidad y las condiciones climáticas. Utilizaremos una versión simplificada del conjunto de datos, que te presento a continuacion:

In [1]:
import pandas as pd

# load the training dataset
bike_data = pd.read_csv('daily-bike-share.csv')
bike_data.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,rentals
0,1,1/1/2011,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331
1,2,1/2/2011,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131
2,3,1/3/2011,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120
3,4,1/4/2011,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108
4,5,1/5/2011,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82


Este es un dataset muy usado en cursos introductorios de machine learning, especialmente en entornos Microsoft (como Azure ML), y contiene datos históricos de uso de bicicletas compartidas.

### Contenido del dataset daily-bike-share.csv

Este dataset contiene registros diarios del número de bicicletas rentadas en un sistema de bike sharing, junto con variables meteorológicas y temporales.

Columna	Descripción
- **day**	Fecha en formato YYYY-MM-DD.
- **season**	Estación del año (1: invierno, 2: primavera, 3: verano, 4: otoño).
- **year**	0 para 2011, 1 para 2012.
- **month**	Número del mes (1-12).
- **holiday**	1 si es día festivo, 0 si no.
- **weekday**	Día de la semana (0: domingo, …, 6: sábado).
- **workingday**	1 si es día laboral, 0 si es fin de semana o feriado.
- **weather**	Estado del clima (1: despejado, 2: nublado, 3: lluvia ligera, 4: tormenta).
- **temp**	Temperatura normalizada (valor entre 0 y 1).
- **atemp**	Sensación térmica normalizada.
- **hum**	Humedad relativa normalizada.
- **windspeed**	Velocidad del viento normalizada.
- **rentals**	Número real de bicicletas rentadas ese día.

In [2]:
bike_data[['temp', 'atemp', 'hum', 'windspeed']]

,temp,atemp,hum,windspeed
0,0.344167,0.363625,0.805833,0.160446
1,0.363478,0.353739,0.696087,0.248539
2,0.196364,0.189405,0.437273,0.248309
3,0.200000,0.212122,0.590435,0.160296
4,0.226957,0.229270,0.436957,0.186900
...,...,...,...,...
726,0.254167,0.226642,0.652917,0.350133
727,0.253333,0.255046,0.590000,0.155471
728,0.253333,0.242400,0.752917,0.124383
729,0.255833,0.231700,0.483333,0.350754


# Variables normalizadas

- **temp**
- **atemp**
- **hum**
- **windspeed**

## ¿Qué significa que esté normalizada entre 0 y 1?

La normalización es una técnica de escalado de valores, usada comúnmente en machine learning para que todos los datos estén en un rango comparable, fórmula:

$$
\text{valor normalizado} = \frac{t - t_{\text{min}}}{t_{\text{max}} - t_{\text{min}}}
$$


